# Data Intelligence Applications: pricing + matching
### Environment Setup

In [2]:
from scipy.optimize import linear_sum_assignment
import numpy as np
from environment import Environment

#Customers
customerParams = zip([200,200,200,200], [1,1,1,1])
#discounts
discounts = np.array([0.0, 0.5, 0.10, 0.25])
#Prices
prices = np.array([250, 400])
#Conversion Rates for item 1
conv_rate1 = [0.3, 0.4, 0.25, 0.45]
#Conversion Rates for item 2 [discountedPrice x Class]
conv_rate2 = np.array([[0.1, 0.2, 0.15, 0.2],
                    [0.15, 0.25, 0.2, 0.25],
                    [0.2, 0.35, 0.25, 0.4],
                    [0.4, 0.45, 0.35, 0.6]])

env = Environment(customerParams, discounts, prices, conv_rate1, conv_rate2)

## Discounts distribution
Our goal now is to find the best possible distribution of discounts for each class. 
This is achieved by matching many possible discount distributions with our 4 classes and maximizing the reward.
Since we are basically bruteforcing the best distribution, we reduced our hypotesis space experimenting with the following settings:
* Uniform fraction distribution
* Promos are decreasingly distributed
* Promos are increasingly distributed

In [3]:
#https://numpy.org/doc/stable/reference/random/generated/numpy.random.dirichlet.html
testDistributions = np.array([[1,1,1,1], [5,3,2,1], [1,2,3,5]]) #Different kinds of distributions we want to test
fractionSamples = 5 #number of fractions we want to sample from each distribution
customers = np.array([100,100,100,100]) #customers for each class --> TODO:use truncnorm

#returns a numpy array [len(testdist) x fractionSamples x 4]
fractions = np.array([[np.random.dirichlet(dist) for x in range(fractionSamples)] for dist in testDistributions])

priceReduction = 1-env.discounts
rewards = np.array([[np.tensordot(env.conv2, (env.prices[1]*priceReduction*sample)*customers, axes=1 ) for sample in frac] for frac in fractions])

rewards



array([[[ 5524.87035702,  7162.36309154, 10656.63014418, 16420.37994255],
        [ 4422.79357165,  5731.58136781,  8039.41072111, 11662.20906135],
        [ 4501.43463118,  6407.37741466,  8366.70241205, 14718.13984538],
        [ 4355.48258431,  5786.74752466,  8111.14574598, 12636.56666212],
        [ 5543.25582518,  7037.53586653, 10864.78709356, 16398.45814538]],

       [[ 4264.41362974,  5904.48230833,  7872.81152657, 13121.05477513],
        [ 4709.53664918,  6151.47561255,  8747.05340729, 13170.76947867],
        [ 4185.9781171 ,  6035.61249525,  8173.62510449, 15175.50035763],
        [ 4539.63934034,  6205.40911527,  8604.11458383, 14316.86548983],
        [ 4215.99270217,  5778.40253114,  7797.07488731, 12776.89316913]],

       [[ 4899.54707652,  6361.03495379,  9143.47850742, 13678.19872522],
        [ 5477.96396622,  7110.73229359, 10382.43940167, 15766.5356496 ],
        [ 4745.86688082,  6317.23414056,  8959.33708079, 14180.01275809],
        [ 4952.44594993,  6434.576

In [4]:
maxGraphs = np.array([np.amax(test)-test for test in rewards])
#for each distribution, calculate the best assignment with Hungarian
assignments = np.array([list(sorted(zip(*linear_sum_assignment(graph)), key = lambda x: x[1])) for graph in maxGraphs])

assignments


array([[[1, 0],
        [2, 1],
        [4, 2],
        [0, 3]],

       [[1, 0],
        [0, 1],
        [3, 2],
        [2, 3]],

       [[0, 0],
        [4, 1],
        [3, 2],
        [1, 3]]])

In [5]:
#For each assignment, find the best one according to the max reward it generates
weights = [sum(rewards[index][np.transpose(assign)[0], np.transpose(assign)[1]]) for index, assign in enumerate(assignments)]
bestDist = np.argmax(weights)
bestDist

0

In [14]:

#Print the corrisponding fractions and the corrisponding reward
print('Best Fraction for Class0: ' + str(fractions[bestDist][assignments[bestDist][0][0]]))
print('Best Fraction for Class1: ' + str(fractions[bestDist][assignments[bestDist][1][0]]))
print('Best Fraction for Class2: ' + str(fractions[bestDist][assignments[bestDist][2][0]]))
print('Best Fraction for Class3: ' + str(fractions[bestDist][assignments[bestDist][3][0]]))
print('Weight sum (max): ' + str(np.max(weights)))

Best Fraction for Class0: [0.15216726 0.60248786 0.11316031 0.13218457]
Best Fraction for Class1: [0.62655298 0.02004227 0.34229143 0.01111331]
Best Fraction for Class2: [0.0849695  0.1277382  0.05221464 0.73507766]
Best Fraction for Class3: [0.16123955 0.01296821 0.2111902  0.61460204]
Weight sum (max): 38115.33802242044
